In [185]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('p03v3irene.csv')
df1 = df1.fillna(0)
df2 = pd.read_csv('p03v3abbe.csv')
df2 = df2.fillna(0)
df1.rename(columns=lambda x: x.strip(), inplace=True)
df2.rename(columns=lambda x: x.strip(), inplace=True)

columns_to_delete = [0, 2, 3]
df1 = df1.drop(df1.columns[columns_to_delete], axis=1)
df2 = df2.drop(df2.columns[columns_to_delete], axis=1)

values_in_df1 = set(df1['Timestamp'])
df2_filtered = df2[df2['Timestamp'].isin(values_in_df1)]
df2 = df2_filtered

df1 = df1.drop(df1.index[22:1000])
df1 = df1.drop(df1.columns[[38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]], axis=1)
df1.rename(columns = {'Unnamed: 41':'Physical support'}, inplace = True)
print(df1)

df2 = df2.drop(df2.index[22:1000])
#df2 = df2.drop(df2.index[0])
df2 = df2.drop(df2.columns[[38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]], axis=1)
print(df2)


   Timestamp Speaker                                          Utterance  \
0   41:38:00  Parent  Are there more cats or alligators? How many ca...   
1   41:43:00   Child                                               1, 2   
2   41:46:00  Parent                               How many alligators?   
3   41:47:00   Child                                                  1   
4   41:48:00  Parent                       So which is more? Kitty cat!   
5   41:54:00  Parent  Gut said pepperoni, pretzels pistachios and pi...   
6   48:35:00  Parent  Oh, here's the flowers. And here's the bees. H...   
7   48:42:00   Child                                     Can you count?   
8   48:43:00  Parent                                              1, 2.   
9   48:51:00  Parent  What should you do when you get to her door? P...   
10  49:23:00   Robot  How many bows are there in the little girl's h...   
11  49:26:00  Parent                              Count. Count with me.   
12  53:39:00   Robot  Wha

In [183]:
#COMPARE
print(df1.columns == (df2.columns))
df1 = df1.reset_index(drop=True)
df2 = df2.reset_index(drop=True)

print(df1.index)
print(df2.index)
print(df1.index == df2.index)

df_compare = df1.compare(df2, align_axis=1, keep_shape=True, keep_equal=True)

# style func to highlight the cells that are different
def highlight_diff(data, color='yellow'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('self', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

df_compare2 = df_compare[(df1 != df2).any(1)].style.apply(highlight_diff, axis=None)
writer = pd.ExcelWriter('v2_P04_comparison_irene(self)vs.aarna.xlsx', engine='xlsxwriter')
df_compare2.to_excel(writer, sheet_name='Differences')
writer.close()  


[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]
RangeIndex(start=0, stop=22, step=1)
RangeIndex(start=0, stop=22, step=1)
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]


In [184]:
from sklearn.metrics import cohen_kappa_score

# COHEN KAPPA VALUES
arrayColumns = df1.columns
arrayr1all = []
arrayr2all = []

for x in range(3, 37):
    arrayr1all.append(df1[arrayColumns[x]].tolist())
    arrayr2all.append(df2[arrayColumns[x]].tolist())

data = []
arrayColumns = arrayColumns[3:]

for i in range(len(arrayr1all)):
    data.append([arrayColumns[i], cohen_kappa_score(arrayr1all[i], arrayr2all[i])])

arrayr1all_f = np.array(arrayr1all).flatten()
arrayr2all_f = np.array(arrayr2all).flatten()
data.append(['Overall Cohen-Kappa Value', cohen_kappa_score(arrayr1all_f, arrayr2all_f)])

summary = pd.DataFrame(data)
summary.to_csv('p04overall_kappa_irene_vs_aarna', sep='\t', index=False)


/Users/justina/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:663: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/Users/justina/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:663: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/Users/justina/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:663: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/Users/justina/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:663: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/Users/justina/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:663: RuntimeWarning: invalid value encountered in true_d

In [124]:
'''
Polynomial_1(c, a0, a1, ... , an)
(1)	 	i ← 1
(2)	 	y ← a0
(3)	 	while i ≤ n
(4)	 	    	y ← y + ai * Power(c, i)
(5)	 	    	i ← i + 1
(6)	 	return y
'''

def polynomial_1(c, arr, n):
    i = 1
    y = arr[0]
    #print(y)
    while i <= n:
        print(y)
        y = y + arr[i]*(c**i)
        i = i+1
    return y

polynomial_1(2, [2, 1, 2, 1, 3], 5)


2
4
12
20
68


IndexError: list index out of range

In [123]:
'''
Polynomial_2(c, a0, a1, ... , an)
(1)	 	i ← 1
(2)	 	y ← an
(3)	 	while i ≤ n
(4)	 	    	y ← y * c + an - i
(5)	 	    	i ← i + 1
(6)	 	return y
'''

def polynomial_2(c, arr, n):
    i = 1
    y = arr[n-1]
    while i <= n:
        print(y)
        y = y * c + arr[n-1-i]
        i = i+1
    return y

polynomial_2(2, [2, 1, 2, 1, 3], 5)

3
7
16
33
68


139